# 04a - Vertex AI > Notebooks - Models Built in Notebooks with Tensorflow

Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

This notebook shows training a model directly within the runtime of the notebook environment.  Then the model is saved and moved to GCS for deployment to a Vertex AI Endpoint for online predictions.  The model training is done with [Tensorflow](https://www.tensorflow.org/), specifically [Keras](https://keras.io/), and was designed to show a neural network approach to logistic regression.  The training data batches are read from BigQuery using [Tensorflow I/O](https://www.tensorflow.org/io).

**Prerequisites:**

-  01 - BigQuery - Table Data Source

**Overview:**

-  Use Python Client for BigQuery
   -  Read the tables schema from BigQuery INFORMATION_SCHEMA
   -  Prepare the feature information for Tensorflow
-  Define a function that remaps the input data into features and target variables where target is one-hot encoded (classification model with 10 classes)
-  Set Tensorflow I/O read session
-  Demonstrate reading a single batch
-  Train a Tensorflow model
   -  Define the model layers
   -  Compile the model
   -  Fit the model
   -  Evaluate the model (loss, accuracy)
   -  Create prediction with the model
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Upload Model
      -  Model - aiplatform.Model.upoad
   -  Create Endpoint
      -  Endpoint - aiplatform.Endpoint.create
   -  Deploy to Endpoint
      -  Endpoint.deploy(model=Model)
   -  Online Predictions
      -  Endpoint.predict
-  Online Predictions with:
   -  REST call
   -  gcloud CLI

**Resources:**

-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)

**Related Training:**

-  todo

---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/slide_17.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/slide_18.png">

---
## Setup

inputs:

In [12]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'fraud'
NOTEBOOK = '04a'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters
EPOCHS = 10
BATCH_SIZE = 100

packages:

In [13]:
from google.cloud import bigquery

from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf

from google.cloud import aiplatform
from datetime import datetime

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [14]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [15]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

environment:

In [16]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get The Schema of The Training Data

In [17]:
query = f"SELECT * FROM {DATANAME}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{DATANAME}_prepped'"
schema = bigquery.query(query).to_dataframe()

In [18]:
schema

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position
0,statmike-mlops,fraud,fraud_prepped,Time,1,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN
1,statmike-mlops,fraud,fraud_prepped,V1,2,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
2,statmike-mlops,fraud,fraud_prepped,V2,3,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
3,statmike-mlops,fraud,fraud_prepped,V3,4,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
4,statmike-mlops,fraud,fraud_prepped,V4,5,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
5,statmike-mlops,fraud,fraud_prepped,V5,6,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
6,statmike-mlops,fraud,fraud_prepped,V6,7,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
7,statmike-mlops,fraud,fraud_prepped,V7,8,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
8,statmike-mlops,fraud,fraud_prepped,V8,9,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
9,statmike-mlops,fraud,fraud_prepped,V9,10,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN


## Number of Classes for the VAR_TARGET?

In [19]:
nclasses = bigquery.query(query = f'SELECT DISTINCT {VAR_TARGET} FROM {DATANAME}.{DATANAME}_prepped WHERE {VAR_TARGET} is not null').to_dataframe()

In [20]:
nclasses

,Class
0,0
1,1


In [21]:
nclasses = nclasses.shape[0]
nclasses

2

## Prepare Inputs For Tensorflow Training

Use the the table schema to prepare the TensorFlow Model:
- Omit unused columns
- Create `feature_columns` for the model
- Define the `dtypes` for TensorFlow

In [22]:
OMIT = VAR_OMIT.split() + ['splits']

selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != VAR_TARGET:
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

# all the columns in this data source are either float64 or int64
output_types = schema[~schema.column_name.isin(OMIT)].data_type.tolist()
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

Define a function that remaps the input data for TensorFlow into:
- features
- `target` - and one_hot encoded for multi-class classification

In [23]:
def transTable(row_dict):
    target=row_dict.pop(VAR_TARGET)
    target = tf.one_hot(tf.cast(target,tf.int64), nclasses)
    target = tf.cast(target, tf.float32)
    return(row_dict, target)

## Use Tensorflow I/O to Read Batches from BigQuery

Setup TensorFlow_IO client > session > table + table.map
- https://www.tensorflow.org/io/api_docs/python/tfio/bigquery/BigQueryClient

In [24]:
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{PROJECT_ID}",
        project_id = PROJECT_ID,
        table_id = f"{DATANAME}_prepped",
        dataset_id = DATANAME,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training

In [25]:
train = bq_reader('TRAIN').parallel_read_rows().map(transTable).shuffle(BATCH_SIZE*10).batch(BATCH_SIZE)
validate = bq_reader('VALIDATE').parallel_read_rows().map(transTable).batch(BATCH_SIZE)
test = bq_reader('TEST').parallel_read_rows().map(transTable).batch(BATCH_SIZE)

2021-09-23 18:51:34.850938: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2299995000 Hz
2021-09-23 18:51:34.851383: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56390fac8300 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-23 18:51:34.851416: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-09-23 18:51:34.851592: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Review a single batch of the train data:

In [26]:
for a, b in train.take(1):
    columns=list(a.keys())
    print('features:\n',columns)
    print('\ntarget:\n',b[0:10])

features:
 ['Amount', 'Time', 'V1', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V2', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9']

target:
 tf.Tensor(
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]], shape=(10, 2), dtype=float32)


---
## Train the Model In The Notebook

Define the Model:

In [27]:
# Simple Logistic Regression
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)

layers = tf.keras.layers.BatchNormalization()(feature_layer_outputs)
layers = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax)(layers)

model = tf.keras.Model(
    inputs = [v for v in feature_layer_inputs.values()],
    outputs = layers
)
opt = tf.keras.optimizers.SGD() #SGD or Adam
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(
    optimizer = opt,
    loss = loss,
    metrics = ['accuracy']
)

In [29]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Amount (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
Time (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
V1 (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
V10 (InputLayer)                [(None, 1)]          0                                            
_______________________________________________________________________________________

Fit the Model:

In [30]:
history = model.fit(train, epochs = EPOCHS, validation_data = validate)

Epoch 1/10
2274/2274 [==============================] - 79s 35ms/step - loss: 0.0617 - accuracy: 0.9831 - val_loss: 0.0135 - val_accuracy: 0.9991
Epoch 2/10
2274/2274 [==============================] - 84s 37ms/step - loss: 0.0088 - accuracy: 0.9992 - val_loss: 0.0104 - val_accuracy: 0.9992
Epoch 3/10
2274/2274 [==============================] - 82s 36ms/step - loss: 0.0070 - accuracy: 0.9992 - val_loss: 0.0094 - val_accuracy: 0.9992
Epoch 4/10
2274/2274 [==============================] - 81s 36ms/step - loss: 0.0062 - accuracy: 0.9993 - val_loss: 0.0088 - val_accuracy: 0.9992
Epoch 5/10
2274/2274 [==============================] - 79s 35ms/step - loss: 0.0060 - accuracy: 0.9992 - val_loss: 0.0083 - val_accuracy: 0.9992
Epoch 6/10
2274/2274 [==============================] - 81s 36ms/step - loss: 0.0057 - accuracy: 0.9992 - val_loss: 0.0079 - val_accuracy: 0.9992
Epoch 7/10
2274/2274 [==============================] - 83s 37ms/step - loss: 0.0052 - accuracy: 0.9992 - val_loss: 0.0077 -

In [34]:
history.history['loss'][-1]

0.0039153494872152805

Evaluate the model with the test data:

In [35]:
loss, accuracy = model.evaluate(test)

290/290 [==============================] - 5s 18ms/step - loss: 0.0038 - accuracy: 0.9994


In [36]:
loss, accuracy = model.evaluate(validate)

285/285 [==============================] - 6s 21ms/step - loss: 0.0057 - accuracy: 0.9992


In [37]:
loss, accuracy = model.evaluate(train)

2274/2274 [==============================] - 77s 34ms/step - loss: 0.0050 - accuracy: 0.9994


Create Prediction from a batch of the test data and review first row:

In [39]:
model.predict(test.take(1))[0]

array([9.9971837e-01, 2.8160628e-04], dtype=float32)

---
## Serving

### Save The Model

In [40]:
model.save(URI)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


2021-09-23 14:27:43.247955: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: gs://statmike-mlops/fraud/models/04a/assets
INFO:tensorflow:Assets written to: gs://statmike-mlops/fraud/models/04a/assets


### Upload The Model

In [41]:
model = aiplatform.Model.upload(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = URI,
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/691911073727/locations/us-central1/models/4311554528416628736/operations/743658812614901760
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/691911073727/locations/us-central1/models/4311554528416628736
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/691911073727/locations/us-central1/models/4311554528416628736')


In [42]:
model.display_name

'04a_fraud_20210923133320'

### Create An Endpoint

In [43]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/691911073727/locations/us-central1/endpoints/3110562375443415040/operations/4582977494948249600
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/691911073727/locations/us-central1/endpoints/3110562375443415040
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/691911073727/locations/us-central1/endpoints/3110562375443415040')


In [44]:
endpoint.display_name

'04a_fraud_20210923133320'

### Deploy Model To Endpoint

In [45]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/691911073727/locations/us-central1/models/4311554528416628736 to Endpoint : projects/691911073727/locations/us-central1/endpoints/3110562375443415040
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/691911073727/locations/us-central1/endpoints/3110562375443415040/operations/5988100578687844352
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/691911073727/locations/us-central1/endpoints/3110562375443415040


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [63]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [64]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,75176,1.235603,0.041383,0.675286,0.836279,-0.675016,-0.657342,-0.154209,-0.067491,0.602617,...,0.088164,0.396205,0.324557,0.182930,-0.017115,0.014979,0.0,0,6f40e111-2131-4031-aef1-268b2daf02a3,TEST
1,112225,-0.285756,0.965688,2.147689,2.838137,1.104026,1.462921,-0.835272,-0.409875,-0.810586,...,-1.222639,0.044689,1.324343,-0.077670,0.128146,0.179651,0.0,0,c9bdeaf0-79d4-4faf-8aaa-14127a0e3513,TEST
2,113420,1.890283,0.241366,-0.165823,4.068924,-0.146807,0.140339,-0.258090,0.084852,-0.056567,...,0.097449,0.019180,0.060621,0.136349,-0.010745,-0.049814,0.0,0,611f4879-490d-4ab5-9e4f-c7944d6165eb,TEST
3,121910,-1.227268,1.555572,1.245848,4.071686,1.154573,2.058276,-1.179951,-2.211140,-2.248168,...,-0.380109,-1.485002,0.237729,0.524141,-0.037543,0.121962,0.0,0,3212444f-1412-49ae-adfa-a1a0ca54d8c5,TEST


In [65]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
#newob

In [66]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [67]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[[0.999718368, 0.000281606306]], deployed_model_id='1405483723553505280', explanations=None)

In [68]:
prediction.predictions[0]

[0.999718368, 0.000281606306]

In [69]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [70]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [71]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.999718368,
      0.000281606306
    ]
  ],
  "deployedModelId": "1405483723553505280"
}


### Get Predictions: gcloud (CLI)

In [72]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999718368, 0.000281606306]]


---
## Remove Resources
see notebook "XX - Cleanup"